# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

## Оглавление

1. [Открытие и изучение файла](#step1)
2. [Предобработка данных](#step2)
    1. [Обработка пропусков](#step2_1)
    2. [Замена типов данных](#step2_2)
    3. [Обработка дубликатов](#step2_3)
    4. [Лемматизация целей кредита](#step2_4)
    5. [Категоризация данных](#step2_5)
    6. [Обработка артефактов](#step2_6)
3. [Анализ влияния параметров на возврат кредита в срок](#step3)
4. [Общий вывод](#step4)

<a id="step1"></a>
## 1. Открытие и изучение файла

Подготовка к лемматизации.

In [1]:
!pip install pymystem3==0.1.10

  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [2]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-10-01 18:48:25--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.245, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-10-01 18:48:27--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  15.70M  4

In [3]:
# Подключение необходимых библиотек
import pandas as pd

# для лемматизации
from pymystem3 import Mystem

Прочитаем данные из файла.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
clients = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1.1 Data_preprocessing/data.csv')

Просмотрим какие есть колонки, типы данных в них, количество строк.

In [6]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Узнаем какие столбцы есть в таблице. Также проверим, что нет лишних пробелов в названиях.

In [7]:
clients.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Выведем первые 10 строк таблицы, чтобы сформировать первичное представление о таблице.

In [8]:
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В таблице 12 колонок и 21525 строк. Названия колонок приведены в правильную форму (на одном языке, без лишних пробелов, в одном регистре). В колонках `days_employed` и `total_income` есть пропущенные значения. Как видно из info, для данных колонок количество пропусков у них одинаковое (количество строк с ненулевыми данными по 19351). Таким образом, можно предположить, что это не случайность. Возможно, что данные обрабатывались вместе (трудовой стаж и ежемесячный доход - связанные параметры), и при обработке что-то пошло не так.

<a id="step2"></a>
## 2. Предобработка данных

<a id="step2_1"></a>
### 2.1. Обработка пропусков

Определим, где есть пустые значения.

In [9]:
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проанализируем столбцы `days_employed` и `total_income`. Поскольку количество пропусков равно, можно предположить, что это одни и те же строки. Проверим данное предположение.

In [10]:
clients[clients['days_employed'].isnull() & clients['total_income'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Действительно, строки одни и те же. Предположение верно - пропуски взаимосвязаны. При следующей выгрузке данных необходимо обратить на это внимание. Теперь необходимо их заполнить.

Сначала рассмотрим столбец `total_income` (ежемесячный доход). Данный параметр более всего зависит от уровня образования.

In [11]:
clients['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Для того, чтобы корректно работать с параметром `education`, приведем все значения в этом столбце к нижнему регистру.

In [12]:
clients['education'] = clients['education'].str.lower()

Найдем средние значения ежемесячного дохода для каждого уровня образования и сохраним их в отдельную таблицу.

In [13]:
mean_education_income = clients.groupby('education')['total_income'].mean()
mean_education_income

education
высшее                 207142.515219
начальное              132155.513626
неоконченное высшее    181534.022774
среднее                153715.643971
ученая степень         174750.155792
Name: total_income, dtype: float64

Теперь можно заполнить пропуски. Для этого подойдут посчитанные средние значения по каждому уровню образования.

In [14]:
clients.loc[clients['total_income'].isnull(), 'total_income'] = clients['education'].map(mean_education_income)

Cтаж работы зависит скорее от возраста, и для каждого человека будет высчитываться индивидуально. Предположим, что человек начал работать в 18 лет. Также, было замечено, что в данной колонке хранятся странные значения - обработаем их позже, а пока добавим в таблицу значение количества дней со знаком минус.

In [15]:
clients.loc[clients['days_employed'].isnull(), 'days_employed'] = (clients['dob_years'] - 18) * 365 * (-1)

Финальная проверка, что во всех столбцах, все значения заполнены

In [16]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод

Пропуски присутствуют в двух колонках - `days_employed` и `total_income` - причем они в одних и тех же строках. Скорее всего они возникли при обработке первоначальных данных. На данном этапе все пропущенные значения были заполнены. Значения в столбце `total_income` (ежемесячный доход) зависят от уровня образования, поэтому данные были сгруппированы по данному параметру, и далее были посчитанны средние значения дохода для каждой группы. Стаж зависит от возраста, в данном случае было решено считать стаж, из расчета на то, что клиент начал работать в 18 лет. Данное число переведено в дни, и добавлено в таблицу со знаком минус (из-за особенностей значений в данном столбце). В дальнейшем значения этого столбца будут обработаны на адекватность.

<a id="step2_2"></a>
### 2.2. Замена типа данных

`days_employed` — общий трудовой стаж в днях -> количество дней всегда должно быть целым числом.

In [17]:
clients['days_employed'] = clients['days_employed'].astype(int)

`total_income` - ежемесячный доход -> могут быть сотые доли денег, но в данном случае лучше сократить дробную часть.

In [18]:
clients['total_income'] = clients['total_income'].astype(int)

Проверка соответствия значений в столбце и их типу данных

In [19]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

Для столбцов `days_employed` и `total_income` были изменены типы данных: float64 -> int64. Был использован метод astype(), потому что это один из способов перевести тип именно в int.

<a id="step2_3"></a>
### 2.3. Обработка дубликатов

Стандартным методом получаем количество дубликатов.

In [20]:
clients.duplicated().sum()

71

Сначала приведем все столбцы со строковыми значениями к нижнему регистру. *(Учтем, что значения в столбце education уже приведены к нижнему регистру)*

In [21]:
str_columns = ['family_status', 'gender', 'income_type', 'purpose']
clients[str_columns] = clients[str_columns].apply(lambda x: x.str.lower())

In [22]:
clients.duplicated().sum()

71

Проверка, какие строки дубликаты есть в таблице.

In [23]:
clients_dupl = clients.copy()
clients_dupl['dupl_status'] = clients_dupl.duplicated()
clients_dupl.loc[clients_dupl['dupl_status'] == True].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dupl_status
2849,0,-8395,41,среднее,1,женат / замужем,0,f,сотрудник,0,153715,покупка жилья для семьи,True
3290,0,-14600,58,среднее,1,гражданский брак,1,f,пенсионер,0,153715,сыграть свадьбу,True
4182,1,-5840,34,высшее,0,гражданский брак,1,f,сотрудник,0,207142,свадьба,True
4851,0,-15330,60,среднее,1,гражданский брак,1,f,пенсионер,0,153715,свадьба,True
5557,0,-14600,58,среднее,1,гражданский брак,1,f,пенсионер,0,153715,сыграть свадьбу,True
6312,0,-4380,30,среднее,1,женат / замужем,0,m,сотрудник,0,153715,строительство жилой недвижимости,True
7808,0,-14235,57,среднее,1,гражданский брак,1,f,пенсионер,0,153715,на проведение свадьбы,True
7921,0,-16790,64,высшее,0,гражданский брак,1,f,пенсионер,0,207142,на проведение свадьбы,True
7938,0,-19345,71,среднее,1,гражданский брак,1,f,пенсионер,0,153715,на проведение свадьбы,True
8583,0,-14600,58,высшее,0,не женат / не замужем,4,f,пенсионер,0,207142,дополнительное образование,True


Удаляем дубликаты.

In [24]:
clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

0

#### Вывод

Для нахождения дубликатов использовался стандартный метод их поиска. Преждевременно значения столбцов со строковыми значениями были приведены к нижнему регистру. Дубликаты в данной таблице могли появиться, если человек несколько раз подавал заявку на кредит.

<a id="step2_4"></a>
### 2.4. Лемматизация целей кредита

Проведем лемматизация для колонки `purpose`.

Получим лемматизатор для слов на русском.

In [25]:
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


Подготовим колонку для лемматизированных строк.

In [26]:
clients['lemm_purpose'] = ''

Реализуем функцию, чтобы получать строки после обработки лемматизатором.

In [27]:
def set_lemm_purpose(row):
    return ' '.join(m.lemmatize(row['purpose']))

Проверим работу лемматизатора на первой строке.

In [28]:
print(clients.loc[0, 'purpose'])
print(m.lemmatize(clients.loc[0, 'purpose']))

покупка жилья
['покупка', ' ', 'жилье', '\n']


Выполним непосредственно лемматизацию.

In [29]:
%%time

clients['lemm_purpose'] = clients.apply(set_lemm_purpose, axis=1)

CPU times: user 2.41 s, sys: 631 ms, total: 3.04 s
Wall time: 5.29 s


Посмотрим в созданной колонке уникальные значения и их количество.

In [30]:
clients['lemm_purpose'].value_counts()

автомобиль \n                                    972
свадьба \n                                       791
на   проведение   свадьба \n                     768
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         661
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 651
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          624
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

Проанализировав вывод, видно что некоторые цели очень похожи по формулировке, а значит их можно выделить в одну группу. Таким образом, выделим все цели по группам: *автомобиль*, *свадьба*, *недвижимость*, *жилье*, *образование*. Заменим в столбце `lemm_purpose` различные цели на выделенные нами группы.

In [31]:
def modify_lemm_purpose(row):
    if 'автомобиль' in row['lemm_purpose']:
        return 'автомобиль'
    if 'свадьба' in row['lemm_purpose']:
        return 'свадьба'
    if 'недвижимость' in row['lemm_purpose']:
        return 'недвижимость'
    if 'жилье' in row['lemm_purpose']:
        return 'жилье'
    if 'образование' in row['lemm_purpose']:
        return 'образование'

In [32]:
clients['lemm_purpose'] = clients.apply(modify_lemm_purpose, axis=1)

Проверим результат.

In [33]:
clients['lemm_purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: lemm_purpose, dtype: int64

Лемматизиция призвана распознать неявные дубликаты. Удалим столбец `purpose`, так как мы уже перенесли информацию в колонку `lemm_purpose`. После проверим, появились ли дубликаты в связи с последними манипуляциями.

In [34]:
clients = clients.drop('purpose', axis=1)

In [35]:
clients.duplicated().sum()

244

In [36]:
clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

0

#### Вывод

Лемматизация проводилсь по столбцу `purpose` (цель получения кредита). Был добавлен новый столбец `lemm_purpose`. Далее были проанализированы значения в этом столбце и выделены по смыслу общие группы целей, которыми были заменены предыдущие значения. Леммы выбирались исходя из смысла строк, полученных после лемматизации. Были проанализированы и выбраны главные слова в строках. Например, "операция   с   жилье" и "покупка   жилье   для   сдача", главным словом здесь является жилье. Именно оно является объектом для кредита. Не важно, что клиент будет делать с целевым объектом, главное сам объект.

<a id="step2_5"></a>
### 2.5. Категоризация данных

В таблице есть дублируемая информация, например столбцы `education` (уровень образования клиента) и `education_id` (идентификатор уровня образования).

In [37]:
education_table = clients[['education_id', 'education']]
education_table.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


В новой таблице есть дубликаты - удалим.

In [38]:
education_table = education_table.drop_duplicates().reset_index(drop=True)
education_table

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Также столбцы `family_status` (семейное положение) и `family_status_id` (идентификатор семейного положения) дублируют информацию.

In [39]:
family_status_table = clients[['family_status_id', 'family_status']]
family_status_table = family_status_table.drop_duplicates().reset_index(drop=True)
family_status_table

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [40]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,lemm_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба


Таким же образом обработаем столбцы `income_type` и `lemm_purpose`. В этим столбцах хранятся строковые значения, а это увеличивает объем памяти для хранения и может вызвать опечатки при работе.

In [41]:
clients['income_type'].value_counts()

сотрудник          10951
компаньон           5045
пенсионер           3755
госслужащий         1453
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Создадим собственный словарь для типов занятости.

In [42]:
income_type_dict = {
    'сотрудник': 0,
    'компаньон': 1,
    'пенсионер': 2,
    'госслужащий': 3,
    'предприниматель': 4,
    'безработный': 5,
    'студент': 6,
    'в декрете': 7
}

Добавим в таблицу столбец для идентификаторов типов занятости.

In [43]:
clients['income_type_id'] = 0

In [44]:
clients['income_type_id'] = clients['income_type'].map(income_type_dict)

In [45]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,lemm_purpose,income_type_id
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,0
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,0
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,0
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,0
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,2


Выделим типы занятости в отдельную таблицу

In [46]:
income_type_table = clients[['income_type_id', 'income_type']]
income_type_table = income_type_table.drop_duplicates().reset_index(drop=True)
income_type_table

,income_type_id,income_type
0,0,сотрудник
1,2,пенсионер
2,1,компаньон
3,3,госслужащий
4,5,безработный
5,4,предприниматель
6,6,студент
7,7,в декрете


Сделаем то же самое для столбца `lemm_purpose`

In [47]:
purpose_dict = {
    'недвижимость': 0,
    'жилье': 1,
    'автомобиль': 2,
    'образование': 3,
    'свадьба': 4
}

In [48]:
clients['purpose_id'] = 0

In [49]:
clients['purpose_id'] = clients['lemm_purpose'].map(purpose_dict)

In [50]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,lemm_purpose,income_type_id,purpose_id
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,0,1
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,0,2
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,0,1
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,0,3
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,2,4


In [51]:
purpose_table = clients[['purpose_id', 'lemm_purpose']]
purpose_table = purpose_table.drop_duplicates().reset_index(drop=True)
purpose_table

,purpose_id,lemm_purpose
0,1,жилье
1,2,автомобиль
2,3,образование
3,4,свадьба
4,0,недвижимость


Удалим лишние столбцы в первоначальной таблице: `education`, `family_status`, `income_type`, `lemm_purpose`

In [52]:
clients = clients.drop(['education', 'family_status', 'income_type', 'lemm_purpose'], axis=1)

In [53]:
clients.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_id
0,1,-8437,42,0,0,f,0,253875,0,1
1,1,-4024,36,1,0,f,0,112080,0,2
2,0,-5623,33,1,0,m,0,145885,0,1
3,3,-4124,32,1,0,m,0,267628,0,3
4,0,340266,53,1,1,f,0,158616,2,4


#### Вывод

При категоризации данных необходимо учитывать, что длинные и сложные значения в столбцах могут привести к ошибкам при запросах к таблице или непосредственно работы с ней. Нет необходимости дублировать информацию в одной и той же таблице. Если можно поставить идентификатор к категории, то в основной таблице можно оставить только его, а соотношение идентификатора к категории вынести в отдельную таблицу.

<a id="step2_6"></a>
### 2.6. Обработка артефактов

После обработки таблицы, необходимо также обработать артефакты в данных. Найти значения, которые не отражают действительность, проанализировать их и устранить. Посмотрим внимательнее на столбцы `children`, `days_employed`, `dob_years`, `gender`, `debt`, `total_income`. Остальные параметры уже проверены.

In [54]:
clients['children'].value_counts()

 0     13892
 1      4773
 2      2043
 3       329
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь есть несколько неправдоподобных значений: 20 и -1. Скорее всего это опечатки пользователей при вводе данных. Заменим эти значения на на более вероятные 2 и 1.

In [55]:
clients['children'] = clients['children'].replace(-1, 1)
clients['children'] = clients['children'].replace(20, 2)
clients['children'].value_counts()

0    13892
1     4820
2     2119
3      329
4       41
5        9
Name: children, dtype: int64

Ранее было замечено, что много значений в столбце с общим трудовым стажем в днях - отрицательные. Если посмотреть на диапазон этих значений и проверить адекватность (выразив эти числа в годах), то можно увидеть, что эти значения вполне могут описывать реальность. Возможно, пользователи по ошибке ввели такое значения в ячейку. Можно предположить, что в ячейке уже стояло значение '-' по умолчанию.

In [56]:
print('Count days employed with values below zero:', clients.loc[clients['days_employed'] < 0]['days_employed'].count())

negative_min_days = clients.loc[clients['days_employed'] < 0]['days_employed'].min()
print('Negative minimum of days employed:', -negative_min_days)
print('Negative minimum of years employed:', -negative_min_days // 365)

negative_max_days = clients.loc[clients['days_employed'] < 0]['days_employed'].max()
print('Negative maximum of days employed:', -negative_max_days)
print('Negative maximum of years employed:', -negative_max_days // 365)

Count days employed with values below zero: 17755
Negative minimum of days employed: 20075
Negative minimum of years employed: 55
Negative maximum of days employed: 24
Negative maximum of years employed: 0


Рассмотрим положительные значения этого столбца. Некоторые значения превышают адекватную оценку дней работы.

In [57]:
print('Count days employed with values above zero:', clients.loc[clients['days_employed'] > 0]['days_employed'].count())

positive_min_days = clients.loc[clients['days_employed'] > 0]['days_employed'].min()
print('Positive minimum of days employed:', positive_min_days)
print('Positive minimum of years employed:', positive_min_days // 365)

positive_max_days = clients.loc[clients['days_employed'] > 0]['days_employed'].max()
print('Positive maximum of days employed:', positive_max_days)
print('Positive maximum of years employed:', positive_max_days // 365)

Count days employed with values above zero: 3455
Positive minimum of days employed: 6570
Positive minimum of years employed: 18
Positive maximum of days employed: 401755
Positive maximum of years employed: 1100


Если посмотреть внимательнее, то можно заметить, что большинство таких строк, это те, у кого тип занятости пенсионеры.

In [58]:
clients.loc[clients['days_employed'] > 0]['income_type_id'].value_counts()

2    3446
0       5
5       2
1       2
Name: income_type_id, dtype: int64

In [59]:
income_type_table

,income_type_id,income_type
0,0,сотрудник
1,2,пенсионер
2,1,компаньон
3,3,госслужащий
4,5,безработный
5,4,предприниматель
6,6,студент
7,7,в декрете


In [60]:
clients.loc[(clients['days_employed'] > 0) & (clients['income_type_id'] == 2)]['days_employed'].head()

4     340266
18    400281
24    338551
25    363548
30    335581
Name: days_employed, dtype: int64

Также посмотрим на оставшиеся строки, с положительным значением трудового стажа.

In [61]:
clients.loc[(clients['days_employed'] > 0) & (clients['income_type_id'] != 2)]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_id
1886,0,6570,0,0,4,f,0,207142,0,1
3125,1,337524,31,1,0,m,1,59956,5,1
4051,1,6570,0,1,1,m,0,153715,1,1
4994,0,6570,0,1,0,f,0,153715,1,0
8523,0,6570,0,1,0,f,0,153715,0,0
12296,3,6570,0,1,0,m,0,153715,0,0
13615,0,6570,0,1,1,f,0,153715,0,4
14651,0,395302,45,0,1,f,0,202722,5,1
19557,0,6570,0,1,0,f,0,153715,0,1


Можно заметить, что неадекватное значение дней трудового стажа (очень большое значение) у категории безработных.

Касаемо текущих положительных значений (пенсионеры и пара безработных), значения в столбце со стажем имеют 6 разрядов. Скорее всего старшие два разряда - это указатель на какую-то дополнительную информацию. Они могли появиться из-за неправильной обработки таблицы (к примеру, значение этого показателся склеилось со стажем). Для этих случаев - отбросим старшие разряды, для случаев с отрицательным значением - сделаем их положительными.

In [62]:
def modify_days_employed(row):
    if row['days_employed'] < 0:
        return row['days_employed'] * (-1)
    elif row['days_employed'] > 0 and row['income_type_id'] in [2, 5]:
        return row['days_employed'] % 10000
    else:
        return row['days_employed']

In [63]:
clients['days_employed'] = clients.apply(modify_days_employed, axis=1)

In [64]:
clients.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_id
0,1,8437,42,0,0,f,0,253875,0,1
1,1,4024,36,1,0,f,0,112080,0,2
2,0,5623,33,1,0,m,0,145885,0,1
3,3,4124,32,1,0,m,0,267628,0,3
4,0,266,53,1,1,f,0,158616,2,4


In [65]:
print('Minimum days_employed: ', clients['days_employed'].min())
print('Maximum days_employed: ', clients['days_employed'].max())

Minimum days_employed:  0
Maximum days_employed:  20075


Рассмотрим значение возраста клиента в годах. Видно, что некоторые клиенты не указали свой реальный возраст (он равен 0).

In [66]:
print('Minimum dob_years: ', clients['dob_years'].min())
print('Maximum dob_years: ', clients['dob_years'].max())

Minimum dob_years:  0
Maximum dob_years:  75


Заменим такой возраст на стаж работа человека + 18 лет (возраст с которого можно работать).

In [67]:
clients.loc[clients['dob_years'] == 0, 'dob_years'] = clients['days_employed'] // 365 + 18

In [68]:
print('Modified minimum dob_years: ', clients['dob_years'].min())
print('Modified maximum dob_years: ', clients['dob_years'].max())

Modified minimum dob_years:  18
Modified maximum dob_years:  75


В столбце пол клиента есть одно значение, не подходящее под общепринятые обозначения.

In [69]:
clients['gender'].value_counts()

f      13988
m       7221
xna        1
Name: gender, dtype: int64

Заменим пол на женский, так как он более вероятный.

In [70]:
clients['gender'] = clients['gender'].replace('xna', 'f')

In [71]:
clients['gender'].value_counts()

f    13989
m     7221
Name: gender, dtype: int64

В столбце задолженности - без подозрительных значений.

In [72]:
clients['debt'].value_counts()

0    19470
1     1740
Name: debt, dtype: int64

В столбце с доходом, проверим, что нет отрицательных значний, при этом верхней границы нет.

In [73]:
print('Minimum total_income: ', clients['total_income'].min())
print('Maximum total_income: ', clients['total_income'].max())

Minimum total_income:  20667
Maximum total_income:  2265604


#### Вывод

На данном шаге были проанализированы различные артефакты. Все некорректные значения были обработаны. В итоге мы получили таблицу, пригодную для анализа данных.

<a id="step3"></a>
## 3. Анализ влияния параметров на возврат кредита в срок

### 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Вычислим долю людей невернувших кредит от количества детей.

In [74]:
child_debt_table = clients.groupby('children')['debt'].mean()
child_debt_table

children
0    0.076447
1    0.092324
2    0.095328
3    0.082067
4    0.097561
5    0.000000
Name: debt, dtype: float64

Для клиентов у которых 5 детей получилось, что должников нет. Проверим - выведем на экран всех таких клиентов.

In [75]:
clients.loc[clients['children'] == 5]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,income_type_id,purpose_id
3966,5,8760,42,1,1,m,0,153715,0,2
4382,5,3248,36,1,0,f,0,168460,1,0
7823,5,773,36,1,0,f,0,48772,0,1
15650,5,418,31,1,0,f,0,77552,0,2
15742,5,2286,37,1,0,f,0,256698,0,0
16029,5,387,35,1,1,f,0,126102,3,4
20157,5,268,38,3,0,f,0,212545,0,3
20538,5,2386,35,1,0,f,0,204241,1,1
20848,5,1690,59,1,0,m,0,269068,0,0


#### Вывод

Как видно из таблицы доля невернувших кредит зависит от количества детей у заемщика. Если у клиента есть хотя бы один ребенок, то вероятность не возврата кредита возрастает, с 7.6% до 9.2%. Среди клентов с детьми выделяются те, у кого 3 детей. Они возвращают кредит чаще, по сравнению с другими. При этом, если детей много (больше 4), кредит с большой вероятностью будет возвращен в срок, так как у заемщика высокий уровень ответственности при таком семейном положении. Данный результат говорит прежде всего о том, что при выборе заемщика, лучше всего отдавать предпочтение семьям без детей, а также многодетным семьям. Наличие детей может повлиять на своевременный возврат денег.

### 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выделим данные в отдельную таблицу.

In [76]:
family_debt_table = clients[['family_status_id', 'debt']]

family_debt_table = family_debt_table.merge(family_status_table, on='family_status_id', how='left')
family_debt_table = family_debt_table[['family_status', 'debt']]

In [77]:
family_debt_grouped_table = family_debt_table.groupby('family_status')['debt'].mean()
family_debt_grouped_table

family_status
в разводе                0.071249
вдовец / вдова           0.066456
гражданский брак         0.093969
женат / замужем          0.076556
не женат / не замужем    0.098138
Name: debt, dtype: float64

#### Вывод

Были изучены доли должников в зависимости от семейного положения. Наиболее надежными в этом плане являются клиенты являющиеся вдовцами или вдовами, находящиеся в разводе или наоброт у которых есть семья. Это категории с высокой социальной ответственностью. К клиентам, не состоящих в отношениях или проживающий в гражданском браке, стоит отнестись c дополнительным вниманием, потому что для них доля задолженности по кредиту несколько выше.

### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разобьем значения столбца по квантилям на три равные части (условно: бедные, средний класс, богатые).

In [78]:
income = pd.qcut(clients['total_income'], 3)

С помощью сводной таблицы получим доли должников в зависимости от достатка.

In [79]:
clients.pivot_table('debt', [income], aggfunc='mean')

,debt
total_income,
"(20666.999, 122120.0]",0.081754
"(122120.0, 179036.667]",0.091513
"(179036.667, 2265604.0]",0.072843


#### Вывод

Как и следовало ожидать среди людей "с большим доходом" доля невозвращающих кредит самая маленькая. Наиболее недобросовестные клиенты "со средним достатком". Скорее всего это зависит от того, что они охотнее берут кредиты. Люди "с маленьким достатком", более ответственно подходят к вопросу о займе денег.

### 3.4. Как разные цели кредита влияют на его возврат в срок?

Выделим данные в отдельную таблицу.

In [80]:
purpose_debt_table = clients[['purpose_id', 'debt']]

purpose_debt_table = purpose_debt_table.merge(purpose_table, on='purpose_id', how='left')
purpose_debt_table = purpose_debt_table[['lemm_purpose', 'debt']]

С помощью сводной таблицы получим доли должников в зависимости от цели кредита.

In [81]:
purpose_debt_table.pivot_table('debt', index='lemm_purpose', aggfunc='mean')

,debt
lemm_purpose,
автомобиль,0.094101
жилье,0.069873
недвижимость,0.075719
образование,0.093340
свадьба,0.080659


#### Вывод

Кредиты взятые на жилье или недвижимость людьми возвращаются скорее, так как это первонеобходимые вещи. Автомобиль и образование вещи менее стабильные, что-то может пойти не так, и кредит не будет возвращен вовремя.

<a id="step4"></a>
## 4. Общий вывод

Была получена статистика о платёжеспособности клиентов банка. Были найдены дубликаты, значение приведены к корректным типам данных, проанализированы цели кредита и также выделены категории для дальнейшей работы. Таблица была изучена на наличие исключительных значений (артефактов) - значение в колонке со стажем не совсем корректные в первоначальном виде (можно скорректировать работу тех, кто формирует таблицу). Также было выявлено, что некоторые пользователи не указали свой возраст (стоит выяснить по какой причине люди не хотят его указывать).
Был проведен анализ различных показателей, которые могут влиять на возврат кредита в будущем. При рассмотрении количества детей стоит отдавать предпочтение клиентам без детей или с большим количеством детей (больше 4). Также стоит учитывать текущее семейное положение клиента и не выбирать тех, у кого низкая социальная ответственность (у кого нет партнера или кто живет в гражданском браке и не может его официально зарегистрировать). Естественно уровень заработка влияет на возврат кредитов, и чем он больше, тем предпочтительнее. Указанные цели играют не последнюю роль в вопросе возврата кредита, стоит учитывать наиболее необходимые вещи для клиента, тогда доля возврата в срок будет выше.